# Movies Recomendation 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import warnings

In [3]:
df = pd.read_csv('movies_metadata.csv')

C:\Users\Suyash Singh\AppData\Local\Temp\ipykernel_4800\2089594497.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


In [4]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
df.shape

(45466, 24)

In [9]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [11]:
df.duplicated().sum()

np.int64(13)

In [12]:
df = df.drop_duplicates().reset_index(drop=True)

In [14]:
df = df[['title','overview', 'genres', 'tagline','vote_average', 'vote_count', 'popularity']]

In [16]:
df.head()

,title,overview,genres,tagline,vote_average,vote_count,popularity
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,7.7,5415.0,21.946943
1,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,6.9,2413.0,17.015539
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,11.7129
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,6.1,34.0,3.859495
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,5.7,173.0,8.387519


In [18]:
df = df.dropna(subset=['title'])

In [19]:
df['overview'] = df['overview'].fillna('')

In [20]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [21]:

#lamda expression to extract genre names from the genres column
import ast
df['genres'] = df['genres'].apply(lambda x:" ".join([i['name'] for i in ast.literal_eval(x)] ) )

In [23]:
df.head()

,title,overview,genres,tagline,vote_average,vote_count,popularity
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,NaN,7.7,5415.0,21.946943
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,2413.0,17.015539
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,11.7129
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,34.0,3.859495
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,173.0,8.387519


In [24]:

df['tagline'] = df['tagline'].fillna('')

In [26]:
df.isnull().sum()

title           0
overview        0
genres          0
tagline         0
vote_average    0
vote_count      0
popularity      0
dtype: int64

In [27]:
df['tags'] = df['overview'] + ' ' + df['genres'] + ' ' + df['tagline']

In [28]:
df.head()


,title,overview,genres,tagline,vote_average,vote_count,popularity,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,,7.7,5415.0,21.946943,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,2413.0,17.015539,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,11.7129,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,34.0,3.859495,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,173.0,8.387519,Just when George Banks has recovered from his ...


In [32]:
df['tags'][2311]

'In a twenty-year career marked by obsessive secrecy, brutality and meticulous planning, Cahill netted over £40 million. He was untouchable - until a bullet from an IRA hitman ended it all. Drama Action Crime The extraordinary true story of the rise and fall of the gangster, Martin Cahill.'

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [34]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Suyash
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\Suyash
[nltk_data]     Singh\AppData\Roaming\nltk_data...


True

In [35]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [36]:
def preprocess_text(text):
  #lowercase
  text = str(text).lower()
  #removed the punctuations
  text = re.sub(r'[^a-zA-Z\s]', '', text)

  words = text.split()
  #remove stopwords
  words = [word for word in words if word not in stop_words]

  #lemmatize
  words = [lemmatizer.lemmatize(word) for word in words]

  return " ".join(words)



In [37]:
df['tags'] = df['tags'].apply(preprocess_text)

In [38]:
df = df.reset_index(drop = True)

In [39]:
indices = pd.Series(df.index,index = df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45442
Century of Birthing            45443
Betrayal                       45444
Satan Triumphant               45445
Queerama                       45446
Length: 45447, dtype: int64

In [48]:
# Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
tfidf = TfidfVectorizer(max_features=50000,ngram_range=(1,2),stop_words='english')

In [42]:
tfidf_matrix = tfidf.fit_transform(df['tags'])

In [43]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1548114 stored elements and shape (45447, 50000)>

In [44]:
# cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
def recommend(title, n = 10):
  if title not in indices:
    return ['Movie not found']

  idx = indices[title]
  sim_score = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
  similar_idx = sim_score.argsort()[::-1][1:n+1]
  return df['title'].iloc[similar_idx]



In [50]:
recommend('Avatar 2',50)

14119                                 The Three Musketeers
28403                               France société anonyme
20372                                               Désiré
43983                                  Nightmare City 2035
7472                                  Frank Herbert's Dune
17491                                        Almighty Thor
26553                                       Thor: Ragnarok
22808                                             Ultraman
37310                                         On the Comet
20708                                          Dinosaurus!
26556                                       Doctor Strange
26133                            The Magnificent Gladiator
40600                                         Maximum Ride
13806                            The Land That Time Forgot
19310                                   Operación Cannabis
9626             Beastmaster 2: Through the Portal of Time
9949                                    A Trip to the Mo

In [47]:
import pickle

pickle.dump(tfidf_matrix,open('tfidf_matrix.pkl','wb'))

pickle.dump(indices,open('indices.pkl','wb'))

df.to_pickle('df.pkl')

In [51]:
pickle.dump(tfidf,open('tfidf.pkl','wb'))